In [1]:
from pyspark.context import SparkContext
from pyspark import SparkConf,sql
from pyspark.sql.session import SparkSession
import findspark
import pymysql
import pymysql.cursors
import pandas as pd
from pyspark.sql.functions import *

In [11]:
product_path='D:/PROJECTS/Recomendation_System_FP/Code/Database/product.csv'
cosin_result_path2='D:/PROJECTS/Recomendation_System_FP/Code/cosin_similarity_registry/collection_tag_FE.csv'#'D:/PROJECTS/Recomendation_System_FP/cosin_simmilarity1.csv'
cosin_result_path1='D:/PROJECTS/Recomendation_System_FP/cosin_simmilarity.csv'

In [3]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .getOrCreate()



In [12]:
Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)

cosin_result_path_df1=spark.read.format('csv').option('header',True) \
.load(cosin_result_path1)

cosin_result_path_df2=spark.read.format('csv').option('header',True) \
.load(cosin_result_path2)

In [72]:
cosin_result_path_df1=cosin_result_path_df1.select('*',round('cosin_score',3).alias('cosin_score_1')).withColumnRenamed('Product_A','original_product').withColumnRenamed('Product_B','matched_product')

In [73]:
cosin_result_path_df1=cosin_result_path_df1.toPandas()

In [74]:
cosin_result_path_df1=cosin_result_path_df1.drop(columns=['cosin_score','_c0'],axis=1).rename(columns={'cosin_score_1':'cosin_score'})

In [76]:
cosin_result_path_df1.to_csv('validated_cosin_simmilarity.csv')

In [30]:
cosin_result_path_df1=cosin_result_path_df1.select(['original_product','matched_product','round(cosin_score, 3)'])

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `original_product` cannot be resolved. Did you mean one of the following? [` original_product`, `matched_product`, `cosin_score`, `cosin_score_1`, `_c0`].;
'Project ['original_product, matched_product#601, round(cosin_score, 3)#632]
+- Project [_c0#553, cosin_score#554,  original_product#595, matched_product#601, cosin_score_1#588, round(cast(cosin_score#554 as double), 3) AS round(cosin_score, 3)#632]
   +- Project [_c0#553, cosin_score#554,  original_product#595, Product_B#556 AS matched_product#601, cosin_score_1#588]
      +- Project [_c0#553, cosin_score#554, Product_A#555 AS  original_product#595, Product_B#556, cosin_score_1#588]
         +- Project [_c0#553, cosin_score#554, Product_A#555, Product_B#556, round(cast(cosin_score#554 as double), 3) AS cosin_score_1#588]
            +- Relation [_c0#553,cosin_score#554,Product_A#555,Product_B#556] csv


In [55]:
product_title='X FASHIONPASS MARCIE EARRINGS'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df1[cosin_result_path_df1['Product_A']==selected_product_id]
link=[]
for row in recomendation_df_product.collect():
    # if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

In [59]:
product_title='KYRA BELT'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=cosin_result_path_df2[cosin_result_path_df2['Product_A']==selected_product_id]
link=[]
for row in recomendation_df_product.collect():
    if float(row['cosin_score'])>=.80:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

3939,4407,7137,8167,2668,6832,8202,4815,8245,7534,8570,2006,5441,4286,7811,5091,2459,7998,5194,5943,8477,2591,5602,5032,6224,6845,8552,7641,3962,8081,1976,4627,2687,5860,3963,2844,8331,3323,7141,3623,8476,4811,4900,7733,8753,4946,7960,4285,8097,7804,7726,5172,7844,7803,1951,4331,4311,7725,3769,4810,1231,2328,7308,5940,8143,5382,8250,7661,6798,5410,5938,6283,8080,3961,6223,6678,2177,7826,4962,4108,4406,2254,8051,8637,6890,3192,7148,3217,5423,7602,8752,5381,8550,8750,3564,8664,4927,7997,5438,4143,8254,7535,4625,5736,180,4102,1531,316,6432,1450,5201,5719,5741,6975,6231,7619,5939,8316,8076,2549,8075,3759,5424,5422,5368,3878,5405,5288,5611,859,6225,6073,3440,1952,3423,5404,5369,5182,4963,7140,7986,2332,4978,7876,4210,7800,6540,4287,3205,3575,7802,7796,1949,5279,1614,7938,5089,8754,3385,557,8475,2001,915,6421,5858,6610,3749,3725,5281,8487,5126,5171,4145,2399,7172,7660,2707,5185,7943,8751,2114,2029,5131,6265,278,6115,3396,8054,8867,8400,4409,3422,8068,7124,8523,6397,6880,8660,1251
